In [1]:
import re
from abc import ABC, abstractmethod
from os import remove
from collections import namedtuple

import pandas as pd
import requests
from requests.exceptions import HTTPError
from bs4 import BeautifulSoup

In [2]:
# Variáveis iniciais

URL_BASE = 'http://compras.dados.gov.br'
PROXIES = {'http': '10.9.16.1:80', 'https': '10.9.16.1:80'}

In [3]:
UASGS = pd.read_csv('listaUASGs.csv')

In [4]:
def limpa_cpf_cnpj(cpfcnpj):
    return re.sub(r'[./-]', '', cpfcnpj)

In [5]:
def request(uri, params):
    endereco = URL_BASE + ''.join(uri)
    try:
        resp = requests.get(endereco, params, proxies=PROXIES, allow_redirects=True)
    except HTTPError:
        return False
    else:
        return resp

In [67]:
def csv2df(uri, params):
    """
    Converte o arquivo CSV disponível em certa url para um dataframe pandas.
    :param url: string
    :param params: dicionário contendo os parâmetros da url
    :return: dataframe pandas com o conteúdo do arquivo CSV.
    """

    resp = request(uri, params)
    with open('tmp.csv', 'wb') as f:
        f.write(resp.content)
    df = pd.read_csv('tmp.csv')
    
    try:
        remove('tmp.csv')
    except FileNotFoundError as erro:
        print(f'Falha ao escrever o CSV baixado de {uri}.')
    
    return df

In [68]:
class Componente(ABC):
    """Classe abstrata, da qual herdarão as classes Uasg, Pregao e Item."""
    
    @property
    def parte_de(self):
        """Retorna o código do componente de que este é parte."""
        try:
            output = self.dados[-1]
        except TypeError:
            output = 'GDF'
        return output
    
    @abstractmethod
    def partes(self):
        """Retorna dataframe correspondente ao CSV que lista as partes do componente."""
        pass
    
    def __getitem__(self, index):
        try:
            output = self.partes().iloc[index]
        except AttributeError:
            output = 'Instância da classe {} não possui partes.'.format(self.__class__.__name__)
        except IndexError:
            output = 'A {} {} não possui partes.'.format(self.__class__.__name__, self.id)
        return output
    
    def __len__(self):
        try:
            tam = len(self.partes())
        except pd.io.common.EmptyDataError:
            tam = 0
        return tam

In [69]:
class Uasg(Componente):
    """Representa uma UASG, no ComprasNet."""
    
    dados = None
    uri = '/pregoes/v1/pregoes'
    colunas = ('Numero do Pregao',
               'Número portaria',
               'Data portaria',
               'Código processo',
               'Tipo do pregão',
               'Tipo de compra',
               'Objeto do pregão',
               'UASG',
               'Situação do pregão',
               'Data de Abertura do Edital',
               'Data de início da proposta',
               'Data do fim da proposta',
               'Resultados do pregão > uri',
               'Declarações do pregão > uri',
               'Termos do pregão > uri',
               'Orgão do pregão > uri',
               'Itens do pregão > uri')
    
    def __init__(self, id):
        self._id = id
        self._params = {'co_uasg': str(self._id)}
    
    @property
    def id(self):
        return self._id
    
#     @property
#     def parte_de(self):
#         return 'GDF'
    
    @property
    def num_partes(self):
        """Retorna o número de pregões da UASG informado no site."""
        resp = request(self.uri, self._params)
        if resp:
            soup = BeautifulSoup(resp.text, 'html.parser')
            return int(soup.find_all(class_='num-resultados')[0].text.split(' ')[-1])
        return 0

    def _offsets(self):
        """Retorna a lista dos offsets a serem utilizados como parâmetro para download dos CSVs"""
        
        return [i * 500 for i in range(self.num_partes // 500 + 1)]
    
    def partes(self):
        """Retorna dataframe correspondente ao CSV dos pregões da UASG."""
        output = pd.DataFrame(columns = self.colunas)
        if self.num_partes:
            for offset in self._offsets():
                self._params['offset'] = offset
                df = csv2df(self.uri + '.csv', self._params)
                output = output.append(df)
            output['parte_de'] = self.id
        return output
    
    def __repr__(self):
        return f'UASG {self._id}'

In [70]:
u = Uasg(989701)

In [71]:
u[0]

'A Uasg 989701 não possui partes.'

In [72]:
class Pregao(Componente):
    """Representa um pregão, no ComprasNet."""
    
    uri = '/pregoes/doc/pregao/'
    colunas = ('Descrição do item',
               'Quantidade do item',
               'Valor estimado do item',
               'Descrição detalhada do Item',
               'Tratamento diferenciado',
               'Decreto 7174',
               'Margem preferencial',
               'Unidade de fornecimento',
               'Situação do item',
               'Fornecedor vencedor',
               'Valor melhor lance',
               'Valor negociado do item',
               'Propostas do Item da licitação > uri',
               'Termos do pregão > uri',
               'Eventos do Item da licitação > uri')
    
    def __init__(self, dados):
        """A classe é instanciada a partir dos dados do pregão, retornados por um objeto Uasg."""
        self.dados = dados
        self._params = {}
    
    @property
    def id(self):
        pattern = re.compile(r'\d+')
        return pattern.findall(self.dados['Itens do pregão > uri'])[0]
    
#     @property
#     def parte_de(self):
#         return self.dados[-1]
    
    @property
    def num_partes(self):
        """Retorna o número de itens do pregão."""
        end = self.uri + self.id + '/itens'
        resp = request(end, self._params)
        if resp:
            soup = BeautifulSoup(resp.text, 'html.parser')
            return int(soup.find_all(class_='num-resultados')[0].text.split(' ')[-1])
        return 0
    
    def _offsets(self):
        """Retorna a lista dos offsets a serem utilizados como parâmetro para download dos CSVs."""
        
        return [i * 500 for i in range(self.num_partes // 500 + 1)]
    
    def partes(self):
        """Retorna dataframe correspondente ao CSV dos itens do pregão."""
        output = pd.DataFrame(columns = self.colunas)
        end = self.uri + self.id + '/itens.csv'
        for offset in self._offsets():
            self._params['offset'] = offset
            df = csv2df(end, self._params)
            output = output.append(df, sort=False)
        output['parte_de'] = self.id
        return output

    def __repr__(self):
        return f'Pregão {self.id}'

In [73]:
class Item(Componente):
    """Representa um item de um pregão. As partes componentes deste item são as propostas."""
    
    uri = '/pregoes/v1/proposta_item_pregao'
    colunas = ('Descrição do Item',
               'Quantidade de itens',
               'Valor estimado do item',
               'Descrição complementar do item',
               'Tratamento diferenciado',
               'Decreto 7174',
               'Margem preferencial',
               'Unidade de fornecimento',
               'Situação do item',
               'Fornecedor vencedor',
               'Valor menor lance',
               'Número cpf/cnpj fornecedor',
               'Fornecedor proposta',
               'Marca do item',
               'Descrição fabricante do item',
               'Descrição detalhada do item',
               'Porte da empresa',
               'Declaração ME/EPP/COOP',
               'Quantidade itens da proposta',
               'Valor unitário',
               'Valor global',
               'Desconto',
               'Valor com Desconto',
               'Data do registro',
               'Data das declarações',
               'Declaração superveniente',
               'Declaração infantil',
               'Declaração independente',
               'Descrição declaração ciência',
               'Descrição motivo cancelamento',
               'Valor classificado',
               'Valor negociado',
               'Observações',
               'Anexos da proposta > uri')
    
    def __init__(self, dados):
        """A classe é instanciada a partir dos dados do item, retornados por um objeto Pregao."""
        self.dados = dados
    
    @property
    def id(self):
        pattern = re.compile(r'item=(\d+)')
        return pattern.findall(self.dados[-2])[0]
    
    def co_uasg(self):
        pattern = re.compile(r'co_uasg=(\d+)')
        return pattern.findall(self.dados[-3])[0]
    
    def co_pregao(self):
        pattern = re.compile(r'co_pregao=(\d+)')
        return pattern.findall(self.dados[-4])[0]
    
    def nu_pregao(self):
        pattern = re.compile(r'nu_pregao=(\d+)')
        return pattern.findall(self.dados[-3])[0]
    
#     @property
#     def parte_de(self):
#         return self.dados[-1]
    
    @property
    def num_partes(self):
        """Retorna o número de propostas apresentadas para este item."""
        
        params = {'item': self.id, 'co_pregao': self.co_pregao()}
        resp = request(self.uri + '.html', params)
        if resp:
            soup = BeautifulSoup(resp.text, 'html.parser')
            return int(soup.find_all(class_='num-resultados')[0].text.split(' ')[-1])
        return 0
    
    def _offsets(self):
        """Retorna a lista dos offsets a serem utilizados como parâmetro para download dos CSVs."""
        
        return [i * 500 for i in range(self.num_partes // 500 + 1)]
        
    def partes(self):
        """Retorna dataframe correspondente ao CSV das propostas para esse item."""
        
        output = pd.DataFrame(columns = self.colunas)
        if self.num_partes:
            params = {'item': self.id, 'co_pregao': self.co_pregao()}
            for offset in self._offsets():
                params['offset'] = offset
                df = csv2df(self.uri + '.csv', params)
                output = output.append(df)
            output['parte_de'] = self.id
        return output
    
    def eventos(self):
        """Retorna dataframe correspondente ao CSV da lista de eventos relativo a este item."""
        
        uri = '/pregoes/v1/evento_item_pregao'
        params = {'item': self.id}
        df = csv2df(uri + '.csv', params)
        df['Data e hora do evento'] = pd.to_datetime(df['Data e hora do evento'])
        return df
    
    def adj_homologado(self):
        eventos = self.eventos()['Descrição do evento'].values
        return 'Adjudicado' in eventos and 'Homologado' in eventos
    
    def prop_venc(self):
        """Retorna dados da proposta vencedora, se houver adjudicação."""

        Proposta = namedtuple('Proposta', 'nome cnpj valor data_adj')

        if self.adj_homologado():
            df = self.eventos()
            obs = df.loc[df['Descrição do evento'] == 'Adjudicado'][
                'Observação'][0]
            nome_pattern = re.compile(r'Fornecedor:\s(.*?),')
            nome = nome_pattern.findall(obs)[0]
            cnpj_pattern = re.compile(r'CNPJ/CPF:\s(.*?),')
            cnpj = cnpj_pattern.findall(obs)[0]
            cnpj = limpa_cpf_cnpj(cnpj)
            valor_pattern = re.compile(r'R\$\s*(.*)$')
            valor = valor_pattern.findall(obs)[0]
            valor = float(re.sub(r',', '.', valor))
            data_adj = df.loc[df['Descrição do evento'] == 'Adjudicado'][
                'Data e hora do evento'][0]
            data_adj = data_adj.to_pydatetime().date()
            return Proposta(nome, cnpj, valor, data_adj)
        return None
    
    def __repr__(self):
        return f'Item {self.id} (Pregão {self.parte_de})'

In [74]:
class Proposta(Componente):
    """Representa uma proposta apresentada no pregão."""
    
    def __init__(self, dados):
        """A classe é instanciada a partir da proposta, retornados por um objeto Item."""
        self.dados = dados
    
    @property
    def id(self):
        pattern = re.compile(r'co_proposta=(\d+)')
        return pattern.findall(self.dados[-2])[0]
    
    def partes(self):
        return ()
    
    def __repr__(self):
        return f'Proposta de {self.dados["Número cpf/cnpj fornecedor"]}'